In [83]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np

In [84]:
path = "./geo_data_trees.geojson"

In [85]:
with open(path, 'r') as f:
    data = json.load(f)

In [86]:
df = []
for i in range(len(data["features"])):
    props = data["features"][i]["properties"]
    df.append(props)

In [87]:
# Normalize the data
df = pd.json_normalize(df)

In [88]:
df.head(1)

,Tree ID,Name,DBH (cm),Replacement Value (eur),Carbon Storage (kg),Carbon Storage (eur),Gross Carbon Sequestration (kg/yr),Gross Carbon Sequestration (eur/yr),Avoided Runoff (mcube/yr),Avoided Runoff (eur/yr),...,Total Annual Benefits (eur/yr),Oxygen Production (kg/yr),Height (m),Crown Height (m),Crown Width (m),Canopy Cover (m2),Leaf Area (m2),Leaf Biomass (kg),Latitude,Longitude
0,1.0,Acer pseudoplatanus,8,179.43,12,1.92,1.1,0.17,0,0.01,...,0.21,2.8,3.5,1,1.5,1.8,3.5,0.2,46.051599,11.072014


In [89]:
df['Crown Width (m)'].head(1)

0    1.5
Name: Crown Width (m), dtype: object

In [90]:
# Rename columns
df.columns = df.columns.str.strip().str.lower().str.replace('[\s\/]', '_').str.replace('[()]', '')

/tmp/ipykernel_71613/1823111577.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.strip().str.lower().str.replace('[\s\/]', '_').str.replace('[()]', '')
/tmp/ipykernel_71613/1823111577.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.strip().str.lower().str.replace('[\s\/]', '_').str.replace('[()]', '')


In [91]:
# The last row is the total
df[-1:]

,tree_id,name,dbh_cm,replacement_value_eur,carbon_storage_kg,carbon_storage_eur,gross_carbon_sequestration_kg_yr,gross_carbon_sequestration_eur_yr,avoided_runoff_mcube_yr,avoided_runoff_eur_yr,...,total_annual_benefits_eur_yr,oxygen_production_kg_yr,height_m,crown_height_m,crown_width_m,canopy_cover_m2,leaf_area_m2,leaf_biomass_kg,latitude,longitude
12512,NaN,Total,None,23086243,5484735,881216,71714,11522,2270,4317,...,25099,191237.8,None,None,None,382146.7,1241575.9,101770.6,NaN,NaN


In [92]:
# Remove the last row
df = df.drop(df.index[-1])

In [93]:
# Check for null values
df.isnull().sum()

tree_id                                  0
name                                     0
dbh_cm                                   0
replacement_value_eur                    0
carbon_storage_kg                        0
carbon_storage_eur                       0
gross_carbon_sequestration_kg_yr         0
gross_carbon_sequestration_eur_yr        0
avoided_runoff_mcube_yr                  0
avoided_runoff_eur_yr                    0
carbon_avoided_kg_yr                 12512
carbon_avoided_eur_yr                12512
pollution_removal_g_yr                   0
pollution_removal_eur_yr                 0
energy_savings_eur_yr                12512
total_annual_benefits_eur_yr             0
oxygen_production_kg_yr                  0
height_m                                 0
crown_height_m                           0
crown_width_m                            0
canopy_cover_m2                          0
leaf_area_m2                             0
leaf_biomass_kg                          0
latitude   

In [94]:
# Drop empty columns
df.drop(['carbon_avoided_kg_yr', 'carbon_avoided_eur_yr', 'energy_savings_eur_yr'], axis=1, inplace=True)

In [95]:
df.head(1)

,tree_id,name,dbh_cm,replacement_value_eur,carbon_storage_kg,carbon_storage_eur,gross_carbon_sequestration_kg_yr,gross_carbon_sequestration_eur_yr,avoided_runoff_mcube_yr,avoided_runoff_eur_yr,...,total_annual_benefits_eur_yr,oxygen_production_kg_yr,height_m,crown_height_m,crown_width_m,canopy_cover_m2,leaf_area_m2,leaf_biomass_kg,latitude,longitude
0,1.0,Acer pseudoplatanus,8,179.43,12,1.92,1.1,0.17,0,0.01,...,0.21,2.8,3.5,1,1.5,1.8,3.5,0.2,46.051599,11.072014


In [96]:
# display(df.dtypes)

In [97]:
# Convert tree_id to int
df[df.columns[2:]] = df[df.columns[2:]].astype(float)
df['tree_id'] = df['tree_id'].astype(int)
df['name'] = df['name'].astype('string')
# display(df.dtypes)

In [98]:
df.head(1)

,tree_id,name,dbh_cm,replacement_value_eur,carbon_storage_kg,carbon_storage_eur,gross_carbon_sequestration_kg_yr,gross_carbon_sequestration_eur_yr,avoided_runoff_mcube_yr,avoided_runoff_eur_yr,...,total_annual_benefits_eur_yr,oxygen_production_kg_yr,height_m,crown_height_m,crown_width_m,canopy_cover_m2,leaf_area_m2,leaf_biomass_kg,latitude,longitude
0,1,Acer pseudoplatanus,8.0,179.43,12.0,1.92,1.1,0.17,0.0,0.01,...,0.21,2.8,3.5,1.0,1.5,1.8,3.5,0.2,46.051599,11.072014


In [99]:
# df.name = df.name.str.replace('(^|\s)(\S)', lambda x: x.group(1) + x.group(2).upper())

In [100]:
# df.head(1)

In [101]:
# result = df.to_json(orient='split')

In [102]:
# parsed = json.loads(result)
# print(parsed["data"][0])

## Task 1

In [103]:
task1 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
task1_mean = df.groupby('name')['canopy_cover_m2'].mean().reset_index(name='mean_canopy_size')
t1 = task1.merge(task1_mean, on='name')
t1 = t1.round(2)
t1.head(10)

,name,count,mean_canopy_size
0,Celtis australis,1028,42.34
1,Aesculus hippocastanum,581,40.89
2,Carpinus betulus,536,13.55
3,Tilia cordata,509,19.85
4,Platanus x hispanica,502,102.36
5,Tilia x europaea,471,33.70
6,Acer campestre,458,11.88
7,Cupressus,399,4.49
8,Sophora japonica,376,39.66
9,Prunus cerasifera,350,13.30


In [104]:
t1.to_csv('task1.csv', index=False)

In [105]:
# t1.to_json('task1.json', orient='split', index=False)

In [106]:
# import csv
# with open('task1.csv', 'r') as file:
#     reader = csv.reader(file)
#     for row in reader:
#         print(row)

## Task 2

In [107]:
path = './circoscrizioni.json'
with open(path, 'r') as f:
    data = json.load(f)

In [108]:
gdf = gpd.GeoDataFrame.from_features(data["features"])
print(gdf.head())

                                            geometry  numero_cir      area  \
0  POLYGON ((11.15678 46.11661, 11.15687 46.11666...           2  15712181   
1  POLYGON ((11.08026 46.11571, 11.08026 46.11571...           3  36949609   
2  POLYGON ((11.09034 46.07917, 11.09032 46.07899...           4   8663138   
3  POLYGON ((11.17163 46.08095, 11.17161 46.08106...           6  13199398   
4  POLYGON ((11.11135 46.05878, 11.11137 46.05878...          11   3073349   

   perimetro                 nome                                     fumetto  
0      17850                MEANO                 Circoscrizione n. 2 - MEANO  
1      38485              BONDONE               Circoscrizione n. 3 - BONDONE  
2      16403             SARDAGNA              Circoscrizione n. 4 - SARDAGNA  
3      20575           ARGENTARIO            Circoscrizione n. 6 - ARGENTARIO  
4      10962  S.GIUSEPPE-S.CHIARA  Circoscrizione n. 11 - S.GIUSEPPE-S.CHIARA  


In [109]:
# task2 = pd.DataFrame(data, columns=['district', task1['name'][0], task1['name'][1], task1['name'][2], task1['name'][3],
#                                     task1['name'][4]], 'others')

In [110]:
columns_names = np.array(['district', task1['name'][0], task1['name'][1], task1['name'][2], task1['name'][3],
                          task1['name'][4], 'others'])

In [111]:
#  TEST
np.argmax(columns_names == task1['name'][3])

4

In [112]:
# TEST
'test' in columns_names

False

In [113]:
gdf.nome.count()

12

In [114]:
# task2.head()

In [115]:
gdf2 = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [116]:
# gdf.geometry[0].contains(gdf2.geometry[11])

In [117]:
points_within = gdf2.sjoin(gdf, predicate='within')

In [118]:
points_within.head()

,tree_id,name,dbh_cm,replacement_value_eur,carbon_storage_kg,carbon_storage_eur,gross_carbon_sequestration_kg_yr,gross_carbon_sequestration_eur_yr,avoided_runoff_mcube_yr,avoided_runoff_eur_yr,...,leaf_biomass_kg,latitude,longitude,geometry,index_right,numero_cir,area,perimetro,nome,fumetto
0,1,Acer pseudoplatanus,8.0,179.43,12.0,1.92,1.1,0.17,0.0,0.01,...,0.2,46.051599,11.072014,POINT (11.07201 46.05160),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
1,2,Acer pseudoplatanus,8.0,179.43,12.9,2.07,1.1,0.18,0.0,0.01,...,0.3,46.051541,11.072108,POINT (11.07211 46.05154),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
2,3,Acer pseudoplatanus,9.0,227.09,16.2,2.61,1.3,0.20,0.0,0.01,...,0.3,46.051668,11.071959,POINT (11.07196 46.05167),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
4,7,Cupressus,25.0,1110.29,448.2,72.02,11.1,1.78,0.0,0.02,...,1.5,46.052305,11.071550,POINT (11.07155 46.05230),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
5,8,Acer pseudoplatanus,9.0,227.09,16.2,2.61,1.3,0.20,0.0,0.01,...,0.3,46.052002,11.071766,POINT (11.07177 46.05200),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA


In [119]:
points_within['name'].head()

0    Acer pseudoplatanus
1    Acer pseudoplatanus
2    Acer pseudoplatanus
4              Cupressus
5    Acer pseudoplatanus
Name: name, dtype: string

In [120]:
districts = []
for i in range(0, len(gdf.nome)):
    districts.append(gdf.nome[i])

In [121]:
districts = np.array(districts)

In [122]:
np.argmax('MEANO' == districts)

0

In [123]:
('A' in districts)

False

In [124]:
r = np.zeros((gdf.nome.count(), len(columns_names)))
for row in points_within.iterrows():
    index_row = np.argmax(districts == row[1]['nome'])
    if row[1]['name'] in columns_names:
        index = np.argmax(columns_names == row[1]['name'])
        r[index_row, index] += 1
    else:
        r[index_row, 6] += 1

In [125]:
task2 = pd.DataFrame(data=r,
                     columns=columns_names)

In [126]:
for i in range(len(districts)):
    task2['district'][i] = districts[i]

In [127]:
task2.columns[1:]

Index(['Celtis australis', 'Aesculus hippocastanum', 'Carpinus betulus',
       'Tilia cordata', 'Platanus x hispanica', 'others'],
      dtype='object')

In [128]:
task2[task2.columns[1:]] = task2[task2.columns[1:]].applymap(int)
task2.head()

,district,Celtis australis,Aesculus hippocastanum,Carpinus betulus,Tilia cordata,Platanus x hispanica,others
0,MEANO,9,0,2,1,9,185
1,BONDONE,1,21,3,9,2,256
2,SARDAGNA,0,0,0,0,0,38
3,ARGENTARIO,61,9,18,5,12,349
4,S.GIUSEPPE-S.CHIARA,83,405,123,31,53,2329


In [129]:
# task2.iloc[:, 1:]

In [130]:
task2['tot'] = task2.iloc[:, 1:].sum(axis=1)

In [131]:
# task2.head()

In [132]:
task2.to_csv('task2.csv', index=False)

In [133]:
# task3 = task2.transpose(copy=True)

In [134]:
# task3.to_csv('task3.csv', index=True)

In [135]:
df.head()

,tree_id,name,dbh_cm,replacement_value_eur,carbon_storage_kg,carbon_storage_eur,gross_carbon_sequestration_kg_yr,gross_carbon_sequestration_eur_yr,avoided_runoff_mcube_yr,avoided_runoff_eur_yr,...,oxygen_production_kg_yr,height_m,crown_height_m,crown_width_m,canopy_cover_m2,leaf_area_m2,leaf_biomass_kg,latitude,longitude,geometry
0,1,Acer pseudoplatanus,8.0,179.43,12.0,1.92,1.1,0.17,0.0,0.01,...,2.8,3.5,1.0,1.5,1.8,3.5,0.2,46.051599,11.072014,POINT (11.07201 46.05160)
1,2,Acer pseudoplatanus,8.0,179.43,12.9,2.07,1.1,0.18,0.0,0.01,...,3.0,5.0,1.5,1.5,1.8,4.2,0.3,46.051541,11.072108,POINT (11.07211 46.05154)
2,3,Acer pseudoplatanus,9.0,227.09,16.2,2.61,1.3,0.20,0.0,0.01,...,3.4,4.5,1.3,1.5,1.8,4.0,0.3,46.051668,11.071959,POINT (11.07196 46.05167)
3,4,Acer pseudoplatanus,11.0,482.46,25.7,4.13,1.6,0.26,0.0,0.02,...,4.3,5.0,1.5,2.5,4.9,7.1,0.5,46.063778,11.150170,POINT (11.15017 46.06378)
4,7,Cupressus,25.0,1110.29,448.2,72.02,11.1,1.78,0.0,0.02,...,29.5,8.0,2.4,1.5,1.8,6.6,1.5,46.052305,11.071550,POINT (11.07155 46.05230)


In [136]:
data2 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
# aa2_mean = df.groupby('name')['canopy_cover_m2'].mean().reset_index(name='mean_canopy_size')
# a2 = aa2.merge(task1_mean, on='name')
# a2 = a2.round(2)
data2.head(10)
print(data2['name'][:10])

0          Celtis australis
1    Aesculus hippocastanum
2          Carpinus betulus
3             Tilia cordata
4      Platanus x hispanica
5          Tilia x europaea
6            Acer campestre
7                 Cupressus
8          Sophora japonica
9         Prunus cerasifera
Name: name, dtype: string


In [137]:
N = 6
top6 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
# task1_mean = df.groupby('name')['canopy_cover_m2'].mean().reset_index(name='mean_canopy_size')
# t1 = task1.merge(task1_mean, on='name')
# t1 = t1.round(2)
top6_names = top6.head(N)['name']

In [138]:
top6_data = df[['name', 'height_m', 'crown_width_m', 'dbh_cm', 'gross_carbon_sequestration_kg_yr']]
top6_data = top6_data[top6_data['name'].isin(top6_names)]

In [139]:
# top6_names = np.array(top6_names)

In [140]:
# top6_data.sort_values(ascending=True, by='name').head()
## CATEGORICAL SORTING
top6_data['name'] = pd.Categorical(top6_data['name'], categories=top6_names)
top6_data = top6_data.sort_values(by='name')

In [141]:
top6_data.head()

,name,height_m,crown_width_m,dbh_cm,gross_carbon_sequestration_kg_yr
2453,Celtis australis,12.0,8.0,41.0,1.6
2187,Celtis australis,9.0,8.0,29.0,1.3
2186,Celtis australis,8.0,6.0,25.0,1.2
2185,Celtis australis,8.3,6.7,30.1,1.3
2184,Celtis australis,8.3,6.7,30.1,1.3


In [142]:
top6_data.to_csv('top6_data.csv', index=False)

In [143]:
top6_data[top6_data['name'] == 'Celtis australis'].describe()

,height_m,crown_width_m,dbh_cm,gross_carbon_sequestration_kg_yr
count,1028.000000,1028.000000,1028.000000,1028.000000
mean,8.261576,6.699027,30.096984,1.239397
std,3.607014,3.005156,17.220832,0.447360
min,2.000000,1.000000,3.000000,0.100000
25%,5.000000,4.000000,18.000000,0.900000
50%,8.000000,6.700000,28.000000,1.200000
75%,10.000000,9.000000,39.000000,1.500000
max,25.000000,17.000000,166.000000,2.200000


In [144]:
top6_data.describe()

,height_m,crown_width_m,dbh_cm,gross_carbon_sequestration_kg_yr
count,3627.000000,3627.000000,3627.000000,3627.000000
mean,8.363551,6.146705,31.375407,5.879653
std,4.833759,3.941895,27.935480,8.181173
min,1.000000,0.500000,2.000000,0.100000
25%,5.000000,3.000000,13.000000,1.300000
50%,7.000000,6.000000,22.000000,2.300000
75%,10.000000,8.000000,40.000000,6.300000
max,38.000000,26.000000,230.000000,51.600000


In [145]:
# id = 8074
# df[df['tree_id'] == id]

In [146]:
# CO2 assorbita = gros_carbon_sequestration_kg_yr
# diametro tronco = dbh_cm
# altezza = height_m
# diametro chioma = crown_width_m
# copertura foglie = leaf_area_m2
# copertura chioma = canopy_cover_m2
# produzione ossigeno = oxygen_production_kg_yr
# inquinamento rimosso = pollution_removal_g_yr

In [147]:
# top6_data.plot.scatter('dbh_cm', 'gross_carbon_sequestration_kg_yr', c='name', colormap='tab10')

# Assignment 3

In [148]:
# GEOJSON - GEOPANDAS
# path_geo = "./geo_data_trees.geojson"
# gdf3 = gpd.read_file(path_geo)

In [149]:
points_within.head()

,tree_id,name,dbh_cm,replacement_value_eur,carbon_storage_kg,carbon_storage_eur,gross_carbon_sequestration_kg_yr,gross_carbon_sequestration_eur_yr,avoided_runoff_mcube_yr,avoided_runoff_eur_yr,...,leaf_biomass_kg,latitude,longitude,geometry,index_right,numero_cir,area,perimetro,nome,fumetto
0,1,Acer pseudoplatanus,8.0,179.43,12.0,1.92,1.1,0.17,0.0,0.01,...,0.2,46.051599,11.072014,POINT (11.07201 46.05160),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
1,2,Acer pseudoplatanus,8.0,179.43,12.9,2.07,1.1,0.18,0.0,0.01,...,0.3,46.051541,11.072108,POINT (11.07211 46.05154),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
2,3,Acer pseudoplatanus,9.0,227.09,16.2,2.61,1.3,0.20,0.0,0.01,...,0.3,46.051668,11.071959,POINT (11.07196 46.05167),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
4,7,Cupressus,25.0,1110.29,448.2,72.02,11.1,1.78,0.0,0.02,...,1.5,46.052305,11.071550,POINT (11.07155 46.05230),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA
5,8,Acer pseudoplatanus,9.0,227.09,16.2,2.61,1.3,0.20,0.0,0.01,...,0.3,46.052002,11.071766,POINT (11.07177 46.05200),2,4,8663138,16403,SARDAGNA,Circoscrizione n. 4 - SARDAGNA


In [150]:
trento_trees = points_within[['tree_id', 'name', 'oxygen_production_kg_yr', 'canopy_cover_m2', 'geometry', 'nome']]

In [151]:
trento_trees.head()

,tree_id,name,oxygen_production_kg_yr,canopy_cover_m2,geometry,nome
0,1,Acer pseudoplatanus,2.8,1.8,POINT (11.07201 46.05160),SARDAGNA
1,2,Acer pseudoplatanus,3.0,1.8,POINT (11.07211 46.05154),SARDAGNA
2,3,Acer pseudoplatanus,3.4,1.8,POINT (11.07196 46.05167),SARDAGNA
4,7,Cupressus,29.5,1.8,POINT (11.07155 46.05230),SARDAGNA
5,8,Acer pseudoplatanus,3.4,1.8,POINT (11.07177 46.05200),SARDAGNA


In [152]:
trento = gdf[['geometry', 'area','nome']]

In [153]:
trento.head()

,geometry,area,nome
0,"POLYGON ((11.15678 46.11661, 11.15687 46.11666...",15712181,MEANO
1,"POLYGON ((11.08026 46.11571, 11.08026 46.11571...",36949609,BONDONE
2,"POLYGON ((11.09034 46.07917, 11.09032 46.07899...",8663138,SARDAGNA
3,"POLYGON ((11.17163 46.08095, 11.17161 46.08106...",13199398,ARGENTARIO
4,"POLYGON ((11.11135 46.05878, 11.11137 46.05878...",3073349,S.GIUSEPPE-S.CHIARA


In [154]:
# trento_trees.to_file("trento_trees.geojson", driver='GeoJSON')
# trento.to_file("trento.geojson", driver='GeoJSON')

In [155]:
district_count = trento_trees.groupby('nome')['nome'].count().sort_values(ascending=False).reset_index(name='count')

In [156]:
# district_count.head()

In [157]:
trento = trento.join(district_count.set_index(["nome"]), on='nome')

In [158]:
canopy_cover = trento_trees.groupby('nome')['canopy_cover_m2'].sum().sort_values(ascending=False).reset_index(name='canopy_cover')

In [159]:
oxygen = trento_trees.groupby('nome')['oxygen_production_kg_yr'].sum().sort_values(ascending=False).reset_index(name='oxygen')

In [160]:
trento = trento.join(canopy_cover.set_index(["nome"]), on='nome')
trento = trento.join(oxygen.set_index(["nome"]), on='nome')

In [161]:
# trento.head()

In [162]:
trento['density'] = trento['canopy_cover'] / trento['area']

In [163]:
trento.head()

,geometry,area,nome,count,canopy_cover,oxygen,density
0,"POLYGON ((11.15678 46.11661, 11.15687 46.11666...",15712181,MEANO,206,3136.1,2961.1,0.000200
1,"POLYGON ((11.08026 46.11571, 11.08026 46.11571...",36949609,BONDONE,292,5979.6,4559.9,0.000162
2,"POLYGON ((11.09034 46.07917, 11.09032 46.07899...",8663138,SARDAGNA,38,389.2,580.5,0.000045
3,"POLYGON ((11.17163 46.08095, 11.17161 46.08106...",13199398,ARGENTARIO,454,12416.1,6848.3,0.000941
4,"POLYGON ((11.11135 46.05878, 11.11137 46.05878...",3073349,S.GIUSEPPE-S.CHIARA,3024,93333.1,51717.3,0.030369


In [164]:
trento.to_file("trento.geojson", driver='GeoJSON')

In [168]:
trento_trees.head()

,tree_id,name,oxygen_production_kg_yr,canopy_cover_m2,geometry,nome
0,1,Acer pseudoplatanus,2.8,1.8,POINT (11.07201 46.05160),SARDAGNA
1,2,Acer pseudoplatanus,3.0,1.8,POINT (11.07211 46.05154),SARDAGNA
2,3,Acer pseudoplatanus,3.4,1.8,POINT (11.07196 46.05167),SARDAGNA
4,7,Cupressus,29.5,1.8,POINT (11.07155 46.05230),SARDAGNA
5,8,Acer pseudoplatanus,3.4,1.8,POINT (11.07177 46.05200),SARDAGNA


In [169]:
trento_trees[['name', 'geometry', 'nome']].head()

,name,geometry,nome
0,Acer pseudoplatanus,POINT (11.07201 46.05160),SARDAGNA
1,Acer pseudoplatanus,POINT (11.07211 46.05154),SARDAGNA
2,Acer pseudoplatanus,POINT (11.07196 46.05167),SARDAGNA
4,Cupressus,POINT (11.07155 46.05230),SARDAGNA
5,Acer pseudoplatanus,POINT (11.07177 46.05200),SARDAGNA


In [171]:
trento_trees[['name', 'geometry', 'nome']].to_file("trento_trees.geojson", driver='GeoJSON')